In [1]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [3]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [4]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [5]:
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        "26M_train":  {"train_start": "2017-04-01", "train_end": "2019-06-01"},
        "24M_train":  {"train_start": "2017-04-01", "train_end": "2019-04-01"},
        "22M_train":  {"train_start": "2017-04-01", "train_end": "2019-02-01"},
        "20M_train":  {"train_start": "2017-04-01", "train_end": "2018-12-01"},
        "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01"},
        "10M_train":  {"train_start": "2017-04-01", "train_end": "2018-01-25"},
        # "8M_train":   {"train_start": "2017-04-01", "train_end": "2017-10-25"},
        },
}
# scenarios_sensors['5'] = scenarios_sensors['2'].copy()
# scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [6]:
from MLForecastPipeline import *
from lightgbm import LGBMRegressor

2025-03-29 17:20:48,921	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-29 17:20:49,172	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
def assign_custom_weights(df):
    month_weights = {
        1: 5, 2: 4, 3: 3,
        4: 2, 5: 1, 6: 1,
        7: 1, 8: 1, 9: 2,
        10: 3, 11: 4, 12: 5
    }
    df['month'] = df['ds'].dt.month
    df['sample_weight'] = df['month'].map(month_weights)
    return df


In [9]:
from sklearn.linear_model import Ridge, Lasso, SGDRegressor
def split_data(df, scenario, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    test_start = pd.to_datetime(scenario['train_end']) + pd.Timedelta(days=1)
    test_data = df[df[date_col] >= test_start]
    return train_data, test_data

models = {
    # SGD with early stopping (patience-like behavior via tol)
    # "SGD_EarlyStopping": SGDRegressor( random_state=42, early_stopping=True, validation_fraction=0.1, n_iter_no_change=7, tol=1e-4 ),

    # SGD with ElasticNet
    "SGD_ElasticNet": SGDRegressor( penalty='elasticnet', l1_ratio=0.5, alpha=0.001, random_state=42 ),

    # # Full L1 (Lasso-like)
    # "SGD_Lasso": SGDRegressor( penalty='l1', alpha=1, random_state=42 ),

    # # Full L2 (Ridge-like)
    "SGD_Ridge": SGDRegressor( penalty='l2', alpha=1, random_state=42 ),
    "SGDRegressor": SGDRegressor(random_state=42),

    # "XGBRegressor": XGBRegressor(),
    # "LGBMRegressor": LGBMRegressor(),
}

# Define lag transformations
from mlforecast.lag_transforms import *
lag_transforms_options = [
    # {1: [expanding_mean], 7: [rolling_mean_14], 30: [expanding_mean]},
    # {1: [rolling_mean_14], 7: [rolling_mean_30], 30: [expanding_mean]},
    {7: [RollingMean(window_size=7)], 30: [RollingMean(window_size=30)], 60: [RollingMean(window_size=60)], },
    {7: [RollingMean(7), RollingStd(7)], 30: [RollingMean(30)], 60: [ExpandingMean()], 14: [ExponentiallyWeightedMean(alpha=0.3)],},
]

In [10]:
from joblib import Parallel, delayed
import time

def process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    """ Process each scenario independently and save results. """
    print(f'{sensor_name}_{scenario_name}')
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]
    
    train_df, test_df = split_data(formatted_df, scenario)
    optimal_lags_list = get_optimal_lags(train_df, 'y', ratios=ratios)
    target_transforms = get_dynamic_transforms(train_df, remove_boxcox=True)

    results = evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list, winter_weights=True)

    # Save results
    save_results(results, f"results/run_13/{sensor_name}_{scenario_name}.csv")

    return results

def run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    # don't use all cpus (instead all but one)
    results = Parallel(n_jobs=-1, verbose=30)( 
        delayed(process_scenario)(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=ratios)
        for sensor_name, scenarios in scenarios_sensors.items()
        for scenario_name, scenario in scenarios.items()
    )

    return results


In [11]:
results = run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed: 37.8min remaining: 94.5min
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed: 39.8min remaining: 53.1min
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed: 41.4min remaining: 31.1min
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed: 41.6min remaining: 16.7min
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 70.8min finished


In [ ]:
import multiprocessing
cpu_count = multiprocessing.cpu_count()
cpu_count

In [ ]:
# Loop through scenarios and evaluate models
results = []

for sensor_name, scenarios in scenarios_sensors.items():
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]

    for scenario_name, scenario in scenarios.items():

        train_df, test_df = split_data(formatted_df, scenario)

        optimal_lags_list = get_optimal_lags(train_df, 'y', 
                                            # ratios=[1]
                                            ratios=[0.33, 0.66, 1]
                                            #  ratios=[0.25, 0.5, 0.75, 1]
        )
        target_transforms = get_dynamic_transforms(train_df)
        results = evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list, winter_weights=True)

        save_results(results, f"results/run_13/{sensor_name}_{scenario_name}.csv") 